In [1]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [2]:

# Path to dataset
dataset_path = r"C:\Users\brais\Downloads\archive\mammals"

In [3]:
# Image size
IMG_SIZE = (64, 64)

In [4]:
# Load images and labels
X = []
y = []


In [5]:

for class_name in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_path):
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            img = cv2.imread(image_path)
            if img is None:
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, IMG_SIZE)
            img = img.astype('float32') / 255.0  # Normalize
            X.append(img)
            y.append(class_name)

In [6]:
# Convert to numpy arrays
X = np.array(X, dtype=np.float32)
y = np.array(y)

In [7]:
# Encode labels
le = LabelEncoder()
y = le.fit_transform(y)
num_classes = len(le.classes_)

In [8]:
y = keras.utils.to_categorical(y, num_classes)  # One-hot encode labels

In [9]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Define CNN model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

c:\Users\brais\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [22]:
# Train the model
epochs = 25
history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), batch_size=32)

Epoch 1/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.5228 - loss: 1.3403 - val_accuracy: 0.3193 - val_loss: 1.8952
Epoch 2/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.6161 - loss: 1.1438 - val_accuracy: 0.3554 - val_loss: 1.9059
Epoch 3/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.6571 - loss: 1.0673 - val_accuracy: 0.3313 - val_loss: 1.9473
Epoch 4/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.6332 - loss: 1.0468 - val_accuracy: 0.3795 - val_loss: 1.8943
Epoch 5/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.6450 - loss: 0.9870 - val_accuracy: 0.3494 - val_loss: 1.9981
Epoch 6/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.6946 - loss: 0.8443 - val_accuracy: 0.3614 - val_loss: 2.0827
Epoch 7/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.6758 - loss: 0.8010 - val_accuracy: 0.3614 - val_loss: 2.1256
Epoch 8/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.7391 - loss: 0.6519 - val_accuracy: 0.3373 - v

In [23]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3180 - loss: 3.3707
Model Accuracy: 31.33%


In [21]:
def classify_image(image_path):
    img = cv2.imread(image_path)  # Read the image dynamically
    if img is None:
        print(f"Error: Cannot read image at {image_path}")
        return None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, IMG_SIZE)
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    prediction = model.predict(img)
    predicted_label = le.classes_[np.argmax(prediction)]
    print(f"Predicted Class: {predicted_label}")
    return predicted_label

# Example usage
image_path = input("Enter the path of the image to classify: ").strip()
classify_image(image_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Predicted Class: lion


'lion'